# Imports to use COCO dataset

In [1]:
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import skimage.io as io
import math
import random

# Imports to use MediaPipe

In [24]:
import scipy.io
import os
import mediapipe as mp
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import shutil
import time

# Other imports

In [3]:
import numpy as np
import pandas as pd
import requests

# Load annotations from COCO dataset

In [4]:
dataDir = './coco/'
dataType = 'val2017'
annFile = '{}/annotations/instances_{}.json'.format(dataDir, dataType)
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


# Get diferrent categories

In [7]:
ten_images = []

# FRISBEE CATEGORY
catIds_frisbee = coco.getCatIds(catNms=['frisbee'])
imgIds_frisbee = coco.getImgIds(catIds=catIds_frisbee)[:15]

for img_id_frisbee in imgIds_frisbee:
    img_info_frisbee = coco.loadImgs(img_id_frisbee)[0]



four_image_frisbee = coco.loadImgs(imgIds_frisbee[3])[0]
ten_images.append(four_image_frisbee)

nine_image_frisbee = coco.loadImgs(imgIds_frisbee[8])[0]
ten_images.append(nine_image_frisbee)

eleven_image_frisbee = coco.loadImgs(imgIds_frisbee[10])[0]
ten_images.append(eleven_image_frisbee)

fifteen_image_frisbee = coco.loadImgs(imgIds_frisbee[14])[0]
ten_images.append(fifteen_image_frisbee)

#print("10 IMAGES", ten_images)

FRISBEE_CATEGORY = [four_image_frisbee, nine_image_frisbee, eleven_image_frisbee, fifteen_image_frisbee ]


for img_info_frisbee in FRISBEE_CATEGORY:
    img_id_frisbee = img_info_frisbee['id']
    #print("img_id_frisbee", img_id_frisbee)
    annIds = coco.getAnnIds(imgIds=img_id_frisbee, catIds=catIds_frisbee, iscrowd=None)
    anns = coco.loadAnns(annIds)
    # Add annotations to img_info_frisbee
    img_info_frisbee['annotations'] = anns
    #print(img_info_frisbee)
    



#################################################################################
print()

# SNOWBOARD CATEGORY
catIds_snowboard = coco.getCatIds(catNms=['snowboard'])
imgIds_snowboard = coco.getImgIds(catIds=catIds_snowboard)[:15]

for img_id_snowboard in imgIds_snowboard:
    img_info_snowboard = coco.loadImgs(img_id_snowboard)[0]
    


one_image_snowboard = coco.loadImgs(imgIds_snowboard[0])[0]
ten_images.append(one_image_snowboard)

ten_image_snowboard = coco.loadImgs(imgIds_snowboard[9])[0]
ten_images.append(ten_image_snowboard)

fourteen_image_snowboard = coco.loadImgs(imgIds_snowboard[13])[0]
ten_images.append(fourteen_image_snowboard)

print()

# SKATEBOARD CATEGORY
catIds_skateboard = coco.getCatIds(catNms=['skateboard'])
imgIds_skateboard = coco.getImgIds(catIds=catIds_skateboard)[:10]

for img_id_skateboard in imgIds_skateboard:
    img_info_skateboard = coco.loadImgs(img_id_skateboard)[0]
    

five_image_skateboard = coco.loadImgs(imgIds_skateboard[4])[0]
ten_images.append(five_image_skateboard)

ten_image_skateboard = coco.loadImgs(imgIds_skateboard[9])[0]
ten_images.append(ten_image_skateboard)


# BICYCLE CATEGORY
catIds_bicycle = coco.getCatIds(catNms=['bicycle'])
imgIds_bicycle = coco.getImgIds(catIds=catIds_bicycle)[:22]

for img_id_bicycle in imgIds_bicycle:
    img_info = coco.loadImgs(img_id_bicycle)[0]


twenty_one_image_bicycle = coco.loadImgs(imgIds_bicycle[20])[0]
ten_images.append(twenty_one_image_bicycle)

# Store 10 images inside 'extracted_images_from_coco'

In [6]:
true_keypoints_list = []

output_folder= "./extracted_images_from_coco"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i, image_info in enumerate(ten_images):
    img_url = image_info['coco_url']

    I = io.imread(img_url)
    filename = f"image_{i}.jpg"
    output_path = os.path.join(output_folder, filename)
    io.imsave(output_path, I)

# Model MediaPipe

In [37]:
media_pipe_images_dir = "./media_pipe_images_dir"

if not os.path.exists(media_pipe_images_dir):
    os.makedirs(media_pipe_images_dir)

def detect_pose(image_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# CALCULATE THE TIME IT TAKES THE MODEL TO PROCESS EACH IMAGE (INFERENCE TIME)
    
    start_time_mp = time.time()
    
    results = pose.process(image_rgb)

    end_time_mp = time.time()

    processing_time_mp = end_time_mp - start_time_mp
    print(f"Processing time for {os.path.basename(image_path)}: {processing_time_mp} seconds ")
    

# GET PREDICTED COORDENATES:
    predicted_keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            predicted_keypoints.append((landmark.x, landmark.y))
    else:
        pass
    pose.close()
    return predicted_keypoints, processing_time_mp

predicted_keypoints_list_mp = []
total_processing_time_mp = 0
total_images_mp = 0


files = filter(lambda file:file.endswith('.jpg'), os.listdir(output_folder))
for file in files:
    image_file = os.path.join(output_folder, file)
    predicted_keypoints, processing_time_mp = detect_pose(image_file)
    if predicted_keypoints:
        total_processing_time_mp += processing_time_mp
        total_images_mp += 1
        predicted_keypoints_list_mp.append(predicted_keypoints)

# CALCULATE THE AVERAGE PROCESSING TIME

if total_images_mp > 0:
    average_processing_time = total_processing_time_mp / total_images_mp
    print(f"Average processing time just for {total_images_mp} images with pose detection: {average_processing_time} seconds")
else:
    print("None image was process.")

# CALCULATE PROCESSING SPEED

processing_speed_mp = total_images_mp / total_processing_time_mp if total_processing_time_mp > 0 else 0

print(f"Total images processed: {total_images_mp}")
print(f"Processing speed: {processing_speed_mp} images per second")

Processing time for image_0.jpg: 0.19503140449523926 seconds 
Processing time for image_1.jpg: 0.184783935546875 seconds 
Processing time for image_2.jpg: 0.16367864608764648 seconds 
Processing time for image_3.jpg: 0.2024838924407959 seconds 
Processing time for image_4.jpg: 0.20420265197753906 seconds 
Processing time for image_5.jpg: 0.18665599822998047 seconds 
Processing time for image_6.jpg: 0.20148158073425293 seconds 
Processing time for image_7.jpg: 0.1962587833404541 seconds 
Processing time for image_8.jpg: 0.1806929111480713 seconds 
Processing time for image_9.jpg: 0.14948725700378418 seconds 
Average processing time just for 9 images with pose detection: 0.1905855337778727 seconds
Total images processed: 9
Processing speed: 5.246987954319236 images per second
